# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import sys
sys.path.append('../')

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
weather_df = pd.read_csv(".././WeatherPy/weather.csv") 
#weather_df.head()
current = 0
coordinates = []
humidity = []
for index,row in weather_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    Currentcoordinates = (lat, lon)
    coordinates.append(Currentcoordinates)
    humidity.append(row['Humidity'])
 
weather_df = weather_df.drop(['Unnamed: 0'],axis=1)

weather_df.head()

,Temperature,Locations,Humidity,Latitude,Longitude,Wind Speed,Cloudiness
0,57.72,"fakse ladeplads, dk",76,55.22,12.17,11.41,54
1,78.69,"luksuhin, ph",94,14.10,120.88,3.36,20
2,42.35,"levashovo, ru",92,60.10,30.21,13.00,100
3,52.83,"landen, be",81,50.75,5.08,11.41,60
4,77.99,"basiawan, ph",77,6.51,125.51,4.79,100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
figure_layout = {
    'width': '500px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


heatmap_layer = gmaps.heatmap_layer(
    weather_df[['Latitude', 'Longitude']], weights=weather_df['Humidity'],
    max_intensity=20, point_radius=6.0)


fig.add_layer(heatmap_layer)
 
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
minTemp = 60
MaxTemp = 90
WindSpeedLimit = 5

figure_layout = {
    'width': '500px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

weather_df = weather_df.dropna()
idealLocations1 = weather_df.loc[lambda df: weather_df['Temperature'] > minTemp]
idealLocations2 = weather_df.loc[weather_df['Temperature'] < MaxTemp ]
idealLocations2 = idealLocations2.merge(idealLocations1, how='inner')
idealLocationsFinal = idealLocations2
idealLocationsFinal = idealLocationsFinal.loc[idealLocationsFinal['Wind Speed'] > WindSpeedLimit]
heatmap_layer = gmaps.heatmap_layer(
    idealLocationsFinal[['Latitude', 'Longitude']], weights=idealLocationsFinal['Humidity'],
    max_intensity=30, point_radius=7.5)


fig.add_layer(heatmap_layer)

fig

#idealLocationsFinal.head(50)

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
url2 = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=formatted_address,name&locationbias=circle:5000@47.6918452,-122.2226413&key={g_key}"
response2 = requests.get(url2).json()
print(response2['candidates'][0]['formatted_address'])
 
#https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters
#https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=mongolian%20grill&inputtype=textquery&fields=photos,formatted_address,name,opening_hours,rating&locationbias=circle:2000@47.6918452,-122.2226413&key=YOUR_API_KEY


201 5th Ave N, Seattle, WA 98109, United States


In [20]:
hotel_df = weather_df
hotel_df['Hotel Name'] = ''
hotel_df['City'] = ''
hotel_df['Country'] = ''
hotel_df['Formatted Address'] = ''

cities = []
countries = []
hotel_df = hotel_df.rename(columns={"Locations": "Location"})
formattedLocation = []
hotelName = []
for index, row in hotel_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=formatted_address,name&locationbias=circle:5000@{latitude},{longitude}&key={g_key}"
    response = requests.get(url).json()
    name = response['candidates'][0]['name'].title()
    formattedLocation.append(response['candidates'][0]['formatted_address'])
    hotelName.append(name)
    
    location = row['Location']
    location = location.split(",")
    country = location[1].upper()
    city = location[0].title()
    cities.append(city)
    countries.append(country)
   
hotel_df['Hotel Name'] = hotelName
hotel_df['Formatted Address'] = formattedLocation
hotel_df['City'] = cities
hotel_df['Country'] = countries
hotel_df.head()

,Temperature,Location,Humidity,Latitude,Longitude,Wind Speed,Cloudiness,Hotel Name,City,Country,Formatted Address
0,57.72,"fakse ladeplads, dk",76,55.22,12.17,11.41,54,Rødvig Kro & Badehotel,Fakse Ladeplads,DK,"Østersøvej 8, 4673 Rødvig Stevns, Denmark"
1,78.69,"luksuhin, ph",94,14.10,120.88,3.36,20,Taal Vista Hotel,Luksuhin,PH,"Kilometer 60 Emilio Aguinaldo Hwy, Kaybagal So..."
2,42.35,"levashovo, ru",92,60.10,30.21,13.00,100,Levada Park-Hotel,Levashovo,RU,"2-й км Приозерского шоссе, St Petersburg, Leni..."
3,52.83,"landen, be",81,50.75,5.08,11.41,60,Hof Van Stayen,Landen,BE,"Tiensesteenweg 229, 3800 Sint-Truiden, Belgium"
4,77.99,"basiawan, ph",77,6.51,125.51,4.79,100,Datie'S Resthouse Seaside Home Rental,Basiawan,PH,"Santa Maria, Davao Occidental, Philippines"


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Rødvig Kro & Badehotel</dd>\n<dt>City</dt><dd>Fakse Ladeplads</dd>\n<dt>Country</dt><dd> DK</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Taal Vista Hotel</dd>\n<dt>City</dt><dd>Luksuhin</dd>\n<dt>Country</dt><dd> PH</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Levada Park-Hotel</dd>\n<dt>City</dt><dd>Levashovo</dd>\n<dt>Country</dt><dd> RU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hof Van Stayen</dd>\n<dt>City</dt><dd>Landen</dd>\n<dt>Country</dt><dd> BE</dd>\n</dl>\n',
 "\n<dl>\n<dt>Name</dt><dd>Datie'S Resthouse Seaside Home Rental</dd>\n<dt>City</dt><dd>Basiawan</dd>\n<dt>Country</dt><dd> PH</dd>\n</dl>\n",
 '\n<dl>\n<dt>Name</dt><dd>Delta Hotel</dd>\n<dt>City</dt><dd>De Lier</dd>\n<dt>Country</dt><dd> NL</dd>\n</dl>\n',
 "\n<dl>\n<dt>Name</dt><dd>Gamoo Kana'An</dd>\n<dt>City</dt><dd>Dembi Dolo</dd>\n<dt>Country</dt><dd> ET</dd>\n</dl>\n",
 '\n<dl>\n<dt>Name</dt><dd>Domeniile Panciu</dd>\n<dt>City</dt><dd>Buciumeni</dd>\n<dt>Country</dt><dd> RO</dd>\n</dl>\n'

In [22]:
# Add marker layer ontop of heat map


# Display figure

markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…